##imports


In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import  concatenate
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from PIL import Image
import numpy as np

##model

In [ ]:
class PyramidPoolingModule(tf.keras.layers.Layer):
    def __init__(self, num_filters=1, kernel_size=(1, 1), bin_sizes=[1, 2, 3, 6], pool_mode='avg', **kwargs):
        super(PyramidPoolingModule, self).__init__(**kwargs)
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.bin_sizes = bin_sizes
        self.pool_mode = pool_mode
        self.pyramid_pooling = self.build_pyramid_pooling()

    def build_pyramid_pooling(self):
        return PyramidPoolingModule.PyramidPoolingModule(
            num_filters=self.num_filters,
            kernel_size=self.kernel_size,
            bin_sizes=self.bin_sizes,
            pool_mode=self.pool_mode,
        )

    def call(self, inputs):
        return self.pyramid_pooling(inputs)

    class PyramidPoolingModule(tf.keras.layers.Layer):
        def __init__(self, num_filters, kernel_size, bin_sizes, pool_mode, **kwargs):
            super(PyramidPoolingModule.PyramidPoolingModule, self).__init__(**kwargs)
            self.num_filters = num_filters
            self.kernel_size = kernel_size
            self.bin_sizes = bin_sizes
            self.pool_mode = pool_mode
            self.pyramid_layers = []

            for bin_size in bin_sizes:
                self.pyramid_layers.append(
                    layers.Conv2D(
                        filters=num_filters,
                        kernel_size=kernel_size,
                        padding='same',
                        activation='relu'
                    )
                )

        def call(self, inputs):
            outputs = [inputs]

            for i, bin_size in enumerate(self.bin_sizes):
                pooled = tf.keras.layers.AveragePooling2D(pool_size=(bin_size, bin_size))(inputs)
                convolved = self.pyramid_layers[i](pooled)
                resized = tf.image.resize(convolved, tf.shape(inputs)[1:3])
                outputs.append(resized)

            return tf.concat(outputs, axis=-1)
def conv3x3(in_planes, out_planes, stride=1, has_bias=False):
    """
    3x3 convolution with padding
    """
    return tf.keras.layers.Conv2D(
        filters=out_planes,
        kernel_size=(3, 3),
        strides=stride,
        padding='same',
        use_bias=has_bias,
        activation=None
    )

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

class BayarConv2d(tf.keras.layers.Layer):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1, padding=0):
        super(BayarConv2d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.minus1 = tf.ones((self.in_channels, self.out_channels, 1)) * -1.000

        self.kernel = self.add_weight(shape=(self.in_channels, self.out_channels, kernel_size ** 2 - 1),
                                      initializer='random_normal',
                                      trainable=True)

    def bayarConstraint(self):
        kernel_permuted = tf.transpose(self.kernel, perm=[2, 0, 1])
        kernel_sum = tf.reduce_sum(kernel_permuted, axis=0)
        ctr = self.kernel_size ** 2 // 2
        real_kernel = tf.concat([self.kernel[:, :, :ctr], self.minus1, self.kernel[:, :, ctr:]], axis=2)
        real_kernel = tf.reshape(real_kernel, (self.out_channels, self.in_channels, self.kernel_size, self.kernel_size))
        return real_kernel

    def call(self, x):
        x = tf.nn.conv2d(x, self.bayarConstraint(), strides=self.stride, padding='SAME')
        return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_encoder(input_shape=(256, 256, 3)):
    inputs = Input(input_shape)

    resnet50 = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)

    s1 = resnet50.layers[0].output         ## (512 x 512)
    s2 = resnet50.get_layer("conv1_relu").output        ## (256 x 256)
    s3 = resnet50.get_layer("conv2_block3_out").output  ## (128 x 128)
    s4 = resnet50.get_layer("conv3_block4_out").output  ## (64 x 64)
    b1 = resnet50.get_layer("conv4_block6_out").output
    encoder_model = Model(inputs=inputs, outputs=[s1, s2, s3, s4,b1])
    encoder_model.trainable = False

    return encoder_model

def assemblage(input_shape=(256, 256, 3)):
    in_channels= 3
    out_channels = 3
    kernel_size = 3
    stride = 1
    padding = 0
    bayar_conv = BayarConv2d(in_channels, out_channels, kernel_size, stride, padding)
    inputs = Input(input_shape)
    x=bayar_conv(inputs)
    encoder_model = build_encoder(input_shape)
    encoder_layers=encoder_model(inputs)
    encoder_layers_bayer=encoder_model(x)
    num_layers = len(encoder_layers)
    concatenated_layers_list = []
    for layer,layer_bayar in zip(encoder_layers, encoder_layers_bayer):
        concatenated_layers=concatenate([layer,layer_bayar], axis=-1)
        output = conv3x3(in_planes=concatenated_layers.shape[-1], out_planes=layer.shape[-1])(concatenated_layers)
        concatenated_layers_list.append(output)
    s1,s2,s3,s4,b1=concatenated_layers_list
    ppm_output = PyramidPoolingModule()(b1)
    """ Decoder """
    d1 = decoder_block(ppm_output, s4, 256)
    d2 = decoder_block(d1, s3, 128)
    d3 = decoder_block(d2, s2, 64)
    d4 = decoder_block(d3, s1, 32)

    """ Output """

    """ Localization """
    localization_output = Conv2D(1, 1, padding="same", activation="sigmoid",name="localization_output")(d4)

    """ Classification """
    avg_pooled = keras.layers.GlobalAveragePooling2D()(ppm_output)
    classification_output = keras.layers.Dense(
        1, activation="sigmoid", name="classification_output"
    )(avg_pooled)
    model = Model(inputs, outputs=[localization_output,classification_output], name="ResNet50_U-Net")
    return model


In [ ]:
model=assemblage()

94765736/94765736 [==============================] - 1s 0us/step


##data


In [ ]:
!kaggle datasets download -d mohamedbenticha/tsyp-cs-challenge
! unzip "tsyp-cs-challenge.zip"

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: TSYP/final data/SHALLOWFAKE/Casia1.0/CASIA 1.0 groundtruth/Sp/Sp_D_NRN_A_cha0023_cha0026_0510_gt.png  
  inflating: TSYP/final data/SHALLOWFAKE/Casia1.0/CASIA 1.0 groundtruth/Sp/Sp_D_NRN_A_cha0024_ani0058_0417_gt.png  
  inflating: TSYP/final data/SHALLOWFAKE/Casia1.0/CASIA 1.0 groundtruth/Sp/Sp_D_NRN_A_cha0025_cha0024_0511_gt.png  
  inflating: TSYP/final data/SHALLOWFAKE/Casia1.0/CASIA 1.0 groundtruth/Sp/Sp_D_NRN_A_cha0026_art0095_0467_gt.png  
  inflating: TSYP/final data/SHALLOWFAKE/Casia1.0/CASIA 1.0 groundtruth/Sp/Sp_D_NRN_A_cha0028_ani0068_0421_gt.png  
  inflating: TSYP/final data/SHALLOWFAKE/Casia1.0/CASIA 1.0 groundtruth/Sp/Sp_D_NRN_A_cha0029_art0068_0463_gt.png  
  inflating: TSYP/final data/SHALLOWFAKE/Casia1.0/CASIA 1.0 groundtruth/Sp/Sp_D_NRN_A_cha0032_ani0079_0512_gt.png  
  inflating: TSYP/final data/SHALLOWFAKE/Casia1.0/CASIA 1.0 groundtruth/Sp/Sp_D_NRN_A_cha0036_art0090_0465_gt.

In [ ]:
class ValDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, batch_size=8, shuffle=True):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        batch_data = self.data[index * self.batch_size : (index + 1) * self.batch_size]

        images = []
        masks = []
        labels = []

        for _, row in batch_data.iterrows():
            image = load_img(row['image'], target_size=(256, 256))
            image = img_to_array(image) / 255.0
            images.append(image)

            mask = load_img(row['mask'], target_size=(256, 256), color_mode='grayscale')
            mask = img_to_array(mask) / 255.0
            masks.append(mask)

            labels.append(row['class'])

        images = np.array(images)
        masks = np.array(masks)
        labels = np.array(labels)

        return images, [masks, labels]

    def on_epoch_end(self):
        if self.shuffle:
            self.data = self.data.sample(frac=1)
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

class TrainDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, batch_size=32, shuffle=True, augment=True):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.on_epoch_end()
        self.Image_gen = tf.keras.preprocessing.image.ImageDataGenerator(
            brightness_range=[0.2, 1.8]
        )

        if self.augment:
            self.image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=45,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest',
            )

    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        batch_data = self.data[index * self.batch_size: (index + 1) * self.batch_size]

        images = []
        masks = []
        labels = []

        for _, row in batch_data.iterrows():
            image = load_img(row['image'], target_size=(256, 256))
            image = img_to_array(image)
            image = self.Image_gen.random_transform(image)
            image/=255.0


            mask = load_img(row['mask'], target_size=(256, 256))
            mask = img_to_array(mask) / 255.0

            if self.augment:
                augmented = self.image_datagen.random_transform(np.concatenate([image, mask], axis=-1))
                image, mask = np.split(augmented, 2, axis=-1)
                mask=tf.image.rgb_to_grayscale(mask)

            images.append(image)
            masks.append(mask)
            labels.append(row['class'])

        images = np.array(images)
        masks = np.array(masks)
        labels = np.array(labels)

        return images, [masks, labels]

    def on_epoch_end(self):
        if self.shuffle:
            self.data = self.data.sample(frac=1)

In [ ]:
import pandas as pd
sf_train = pd.read_csv("/content/TSYP/train.csv", na_values="None")
sf_train["image"] = "/content/TSYP/" + sf_train["image"]
sf_train["mask"] = "/content/TSYP/" + sf_train["mask"]
sf_train.fillna("/content/TSYP/mask.png", inplace=True)
sf_val = pd.read_csv("/content/TSYP/val.csv", na_values="None")
sf_val["image"] = "/content/TSYP/" + sf_val["image"]
sf_val["mask"] = "/content/TSYP/" + sf_val["mask"]
sf_val.fillna("/content/TSYP/mask.png", inplace=True)
sf_train = sf_train.sample(frac=1).reset_index(drop=True)
sf_val = sf_val.sample(frac=1).reset_index(drop=True)
sf_val = sf_val.drop(sf_val.index[:9949])
sf_train_generator = TrainDataGenerator(sf_train)
sf_val_gen = ValDataGenerator(sf_val)


##compiling


In [ ]:
smooth=1
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
checkpoint = ModelCheckpoint("/content/drive/My Drive/P2M/best-model.h5", save_best_only=True, monitor="val_loss", mode="min", verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_delta=0.0001, mode='auto', verbose=1)

In [ ]:
adam_optimizer = tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999)

In [ ]:
model.compile(optimizer=adam_optimizer, loss=[dice_coef_loss, 'binary_crossentropy'], loss_weights=[0.16, 0.04])

In [ ]:
model.summary()

Model: "ResNet50_U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 bayar_conv2d (BayarConv2d)  (None, 256, 256, 3)          72        ['input_1[0][0]']             
                                                                                                  
 model (Functional)          [(None, 256, 256, 3),        8589184   ['input_1[0][0]',             
                              (None, 128, 128, 64),                  'bayar_conv2d[0][0]']        
                              (None, 64, 64, 256),                                                
                              (None, 32, 32, 512),                                   

In [ ]:
history = model.fit(sf_train_generator, epochs=10, validation_data=sf_val_gen, callbacks=[checkpoint, reduce_lr])

Epoch 1/10
584/584 [==============================] - ETA: 0s - loss: 0.2923 - localization_output_loss: 0.7564 - classification_output_loss: 4.2823

584/584 [==============================] - 1724s 3s/step - loss: 0.2923 - localization_output_loss: 0.7564 - classification_output_loss: 4.2823 - lr: 0.0010
Epoch 2/10
584/584 [==============================] - ETA: 0s - loss: 0.1359 - localization_output_loss: 0.6787 - classification_output_loss: 0.6822

584/584 [==============================] - 1631s 3s/step - loss: 0.1359 - localization_output_loss: 0.6787 - classification_output_loss: 0.6822 - lr: 0.0010
Epoch 3/10
584/584 [==============================] - ETA: 0s - loss: 0.1336 - localization_output_loss: 0.6595 - classification_output_loss: 0.7034

584/584 [==============================] - 1640s 3s/step - loss: 0.1336 - localization_output_loss: 0.6595 - classification_output_loss: 0.7034 - lr: 0.0010
Epoch 4/10
584/584 [==============================] - ETA: 0s - loss: 0.1330 - localization_output_loss: 0.6552 - classification_output_loss: 0.7038

584/584 [==============================] - 1633s 3s/step - loss: 0.1330 - localization_output_loss: 0.6552 - classification_output_loss: 0.7038 - lr: 0.0010
Epoch 5/10
204/584 [=========>....................] - ETA: 17:34 - loss: 0.1333 - localization_output_loss: 0.6641 - classification_output_loss: 0.6756